In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


wocabulary

In [5]:
with open("wizard_of_oz.txt", encoding="utf-8") as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

[' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '·', '\ufeff']


encoder

In [6]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([83,  2, 72, 57, 81, 68, 71, 57, 60,  2, 23, 83,  2, 57, 68, 68, 45, 64,
        71, 74, 76, 59, 77, 76, 75, 31, 70, 57, 58, 68, 61, 60,  2, 23, 62, 57,
        68, 75, 61,  9,  2, 62, 65, 68, 61, 46, 74, 61, 61,  2, 23, 83,  2,  2,
        23, 83,  2, 65, 76, 61, 69, 75,  2, 23, 53, 83,  2, 70, 57, 69, 61,  2,
        23,  2, 44, 31, 27, 30, 39, 31, 11, 69, 60,  2,  9,  2, 72, 57, 76, 64,
         2, 23,  2, 44, 31, 27, 30, 39, 31, 11])


In [8]:
n = int(0.8 * len(data))
train_data = data[:n]
valid_data = data[n:]

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print('when input is', context, 'target is', target)

when input is tensor([83]) target is tensor(2)
when input is tensor([83,  2]) target is tensor(72)
when input is tensor([83,  2, 72]) target is tensor(57)
when input is tensor([83,  2, 72, 57]) target is tensor(81)
when input is tensor([83,  2, 72, 57, 81]) target is tensor(68)
when input is tensor([83,  2, 72, 57, 81, 68]) target is tensor(71)
when input is tensor([83,  2, 72, 57, 81, 68, 71]) target is tensor(57)
when input is tensor([83,  2, 72, 57, 81, 68, 71, 57]) target is tensor(60)
